# Predictive Analytics: Assignment 4.2
#### Joshua Greenert
#### DSC630-T301 Predictive Analytics
#### 1/2/2023

In [1]:
# Import the required libaries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pull in the data to begin preparation.
df_als = pd.read_csv('als_data.csv')
df_als.head(5)

,ID,Age_mean,Albumin_max,Albumin_median,Albumin_min,Albumin_range,ALSFRS_slope,ALSFRS_Total_max,ALSFRS_Total_median,ALSFRS_Total_min,...,Sodium_min,Sodium_range,SubjectID,trunk_max,trunk_median,trunk_min,trunk_range,Urine.Ph_max,Urine.Ph_median,Urine.Ph_min
0,1,65,57.0,40.5,38.0,0.066202,-0.965608,30,28.0,22,...,143.0,0.017422,533,8,7.0,7,0.002646,6.0,6.0,6.0
1,2,48,45.0,41.0,39.0,0.010453,-0.921717,37,33.0,21,...,136.0,0.010453,649,8,7.0,5,0.005386,7.0,5.0,5.0
2,3,38,50.0,47.0,45.0,0.008929,-0.914787,24,14.0,10,...,140.0,0.008929,1234,5,0.0,0,0.008929,6.0,5.0,5.0
3,4,63,47.0,44.0,41.0,0.012111,-0.598361,30,29.0,24,...,138.0,0.012469,2492,5,5.0,3,0.004988,7.0,6.0,5.0
4,5,63,47.0,45.5,42.0,0.008292,-0.444039,32,27.5,20,...,138.0,0.008292,2956,6,4.0,1,0.008489,6.0,5.0,5.0


In [2]:
# Remove any data that is not relevant to the patient’s ALS condition.

In [3]:
# Apply a standard scalar to the data.

In [4]:
# Create a plot of the cluster silhouette score versus the number of clusters in a K-means cluster.

In [ ]:
# Use the plot created in (3) to choose on optimal number of clusters for K-means. Justify your choice.

In [ ]:
# Fit a K-means model to the data with the optimal number of clusters chosen in part (4).

In [ ]:
# Fit a PCA transformation with two features to the scaled data.

In [ ]:
# Make a scatterplot the PCA transformed data coloring each point by its cluster value.

In [ ]:
# Summarize your results and make a conclusion.